In [ ]:
from typing import Annotated, Sequence, TypedDict, List, cast
from dotenv import load_dotenv
from langchain_core.messages import BaseMessage, ToolMessage, AIMessage, SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
import pandas as pd
import json
from pydantic import BaseModel

In [12]:
load_dotenv()

True

In [13]:
class AgentState(TypedDict):
    topic_history: List[str]
    topic: str | None
    tone: str | None

In [ ]:
class TopicResponse(BaseModel):
    topic: str
    tone: str

In [14]:
tools = []

In [ ]:
model = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
topic_model = model.with_structured_output(TopicResponse)

In [ ]:
def get_topic_history(state: AgentState) -> AgentState:
    """Function to get the subject history"""
    
    try:
        df = pd.read_parquet("data/topic_history.parquet")
        topic_history = df['topic_history'].tolist()
    except:
        topic_history = []
    
    return {**state, "topic_history":topic_history}

def get_topic(state: AgentState) -> AgentState:
    """Calls an LLM and ask's for a topic for an horror story"""
    system_prompt = SystemMessage(content=f"""Você é uma assistente focada em sugerir tópicos para histórias de suspense e terror com finais ambíguos e vagos.
                                Responda **exclusivamente** em JSON válido no formato:
                                {{
                                "topic": "...",
                                "tone": "..."
                                }}
                                Onde
                                "topic" (que deve ter como valor o tópico sugerido), ex: "Relato de vigia noturno em estrada rural"
                                "tone" (que deve ter como valor o tom da história), ex: "realista, cansado"
                                É importante que você não repita tópicos já utilizados anteriormente. A seguir esta uma lista de tópicos já utilizados: {state['topic_history']}
                                """)
    
    human_prompt = HumanMessage(content="""Me sugira um tópico para uma história de terror e suspense""")
    
    response = cast(TopicResponse, topic_model.invoke([system_prompt, human_prompt]))

    
    return {**state, "topic":response.topic, "tone":response.tone}
    
